In [2]:
using Pkg

# Instalar si no están
for pkg in ["DataFrames", "Distributions", "GLM", "HypothesisTests", "Lasso"]
    if !Pkg.installed(pkg)
        Pkg.add(pkg)
    end
end

# Cargar librerías
using DataFrames
using Distributions
using GLM
using HypothesisTests
using Lasso

println("✓ Todas las librerías listas")

LoadError: MethodError: no method matching installed(::String)
The function `installed` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  installed()
[0m[90m   @[39m [35mPkg[39m [90mC:\Users\ARIANA\AppData\Local\Programs\Julia-1.11.6\share\julia\stdlib\v1.11\Pkg\src\[39m[90m[4mPkg.jl:784[24m[39m


In [4]:
names = Set([info.name for (_, info) in Pkg.dependencies()])
println(names ∩ Set(["DataFrames","Distributions","GLM","HypothesisTests","Lasso","MLJLinearModels"]))

Set(["DataFrames", "MLJLinearModels", "Lasso", "Distributions", "GLM", "HypothesisTests"])


In [6]:
using Pkg

# Paquetes objetivo
pkgs = ["DataFrames","Distributions","GLM","HypothesisTests","Lasso"]

# Instala o actualiza a la versión compatible (idempotente)
for p in pkgs
    try
        Pkg.add(p)
    catch e
        @warn "No se pudo instalar $(p)" exception=e
    end
end

# Carga básicos
using DataFrames
using Distributions
using GLM
using HypothesisTests

# Lasso: si falla, cae a alternativa
try
    using Lasso
catch
    @info "Paquete Lasso no disponible. Instalando alternativa MLJLinearModels (L1)."
    Pkg.add("MLJLinearModels")
    using MLJLinearModels
end

println("✓ Todas las librerías listas")

   Resolving package versions...
  No Changes to `C:\Users\ARIANA\Project.toml`
  No Changes to `C:\Users\ARIANA\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ARIANA\Project.toml`
  No Changes to `C:\Users\ARIANA\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ARIANA\Project.toml`
  No Changes to `C:\Users\ARIANA\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ARIANA\Project.toml`
  No Changes to `C:\Users\ARIANA\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ARIANA\Project.toml`
  No Changes to `C:\Users\ARIANA\Manifest.toml`


✓ Todas las librerías listas


In [8]:
using Random, Distributions

Random.seed!(42)
n = 1000

# Covariables
X1 = rand(Normal(0,1), n)
X3 = rand(Normal(0,1), n)
X2 = rand(Binomial(1,0.5), n)
X4 = rand(Binomial(1,0.5), n)

# Tratamiento
D  = rand(Binomial(1,0.5), n)

# Error
ϵ  = rand(Normal(0,1), n)

# Outcome
Y = 2 .* D .+ 0.5 .* X1 .- 0.3 .* X2 .+ 0.2 .* X3 .+ ϵ

1000-element Vector{Float64}:
  0.7768113331191083
 -0.2602114854397375
  0.03960482391335382
  0.799281986744393
  1.5031486251776052
 -0.4579783404994724
  0.26599370423462365
 -0.969281186279706
  0.9007027430415538
  3.5970176475017626
  1.2012396854805574
  2.477306063967831
  1.1385671502470969
  ⋮
  0.5055518733172706
  2.0007185862221304
  1.4461040934328426
  0.43756249400442626
  1.9840803937753524
  1.4305308755318278
  1.9296898518403791
  0.2941845243101905
  1.9422745497949871
  0.008397572026267675
 -0.029111158491312544
  3.192716729419783

In [12]:
using DataFrames

df = DataFrame(Y=Y, D=D, X1=X1, X2=X2, X3=X3, X4=X4)

println("="^60)
println("3.1 DATA SIMULATION - COMPLETADO")
println("="^60)
println("Primeras 5 filas:")
display(first(df, 5))
println("Dimensión del DataFrame: ", size(df))

3.1 DATA SIMULATION - COMPLETADO
Primeras 5 filas:


Row,Y,D,X1,X2,X3,X4
,Float64,Int32,Float64,Int32,Float64,Int32
1,0.776811,1,-0.363357,1,-0.01988,1
2,-0.260211,1,0.251737,1,0.218246,0
3,0.0396048,0,-0.314988,1,1.52856,1
4,0.799282,0,-0.311252,0,-0.764316,0
5,1.50315,1,0.816307,1,-2.02765,1


Dimensión del DataFrame: (1000, 6)


In [14]:
using HypothesisTests, Statistics

tratamiento = filter(:D => ==(1), df)
control     = filter(:D => ==(0), df)

function balance_check(var::Symbol)
    x = tratamiento[:, var]; y = control[:, var]
    test = UnequalVarianceTTest(x, y)
    println(string(var), ":")
    println("  Media tratamiento: ", mean(x))
    println("  Media control:     ", mean(y))
    println("  Diferencia:        ", mean(x) - mean(y))
    println("  p-valor:           ", pvalue(test))
    println("-"^40)
end

println("\nBALANCE CHECK - Comparación de medias")
for v in (:X1, :X2, :X3, :X4)
    balance_check(v)
end



BALANCE CHECK - Comparación de medias
X1:
  Media tratamiento: -0.061429513646864324
  Media control:     -0.0588083593394067
  Diferencia:        -0.0026211543074576224
  p-valor:           0.9665323026719614
----------------------------------------
X2:
  Media tratamiento: 0.536144578313253
  Media control:     0.48406374501992033
  Diferencia:        0.052080833293332685
  p-valor:           0.0996962104234329
----------------------------------------
X3:
  Media tratamiento: 0.024307422156750916
  Media control:     0.013996148542409171
  Diferencia:        0.010311273614341745
  p-valor:           0.8725831710469407
----------------------------------------
X4:
  Media tratamiento: 0.5281124497991968
  Media control:     0.5059760956175299
  Diferencia:        0.02213635418166693
  p-valor:           0.484160686513487
----------------------------------------


In [16]:
using GLM, StatsModels, Statistics

modelo_simple = lm(@formula(Y ~ D), df)
println("=== REGRESIÓN SIMPLE (Y ~ D) ===")
display(coeftable(modelo_simple))

β_D_simple  = coef(modelo_simple)[2]
se_D_simple = stderror(modelo_simple)[2]
println("Coef. D (ATE): ", round(β_D_simple, digits=3))
println("SE:            ", round(se_D_simple, digits=3))


=== REGRESIÓN SIMPLE (Y ~ D) ===


LoadError: UndefVarError: `coeftable` not defined in `Main`
Hint: It looks like two or more modules export different bindings with this name, resulting in ambiguity. Try explicitly importing it from a particular module, or qualifying the name with the module it should come from.
Hint: a global variable of this name may be made accessible by importing StatsBase in the current active module Main
Hint: a global variable of this name also exists in GLM.
Hint: a global variable of this name also exists in Lasso.

In [18]:
using GLM, StatsModels, Statistics

modelo_simple = lm(@formula(Y ~ D), df)
println("=== REGRESIÓN SIMPLE (Y ~ D) ===")
display(GLM.coeftable(modelo_simple))

β_D_simple  = coef(modelo_simple)[2]
se_D_simple = stderror(modelo_simple)[2]
println("Coef. D (ATE): ", round(β_D_simple, digits=3))
println("SE:            ", round(se_D_simple, digits=3))


=== REGRESIÓN SIMPLE (Y ~ D) ===


|             |     Coef. | Std. Error |     t | Pr(>|t|) | Lower 95% |  Upper 95% |
|:------------|----------:|-----------:|------:|:---------|:----------|-----------:|
| (Intercept) | -0.156536 |  0.0517306 | -3.03 |   0.0025 | -0.258049 | -0.0550224 |
| D           |  1.96305  |  0.0733049 | 26.78 |   <1e-99 |  1.8192   |  2.1069 |

Coef. D (ATE): 1.963
SE:            0.073


In [20]:
modelo_completo = lm(@formula(Y ~ D + X1 + X2 + X3 + X4), df)
println("=== REGRESIÓN CON CONTROLES ===")
display(GLM.coeftable(modelo_completo))

β_D_comp  = coef(modelo_completo)[2]
se_D_comp = stderror(modelo_completo)[2]
println("Coef. D (ATE): ", round(β_D_comp, digits=3))
println("SE:            ", round(se_D_comp, digits=3))


=== REGRESIÓN CON CONTROLES ===


|             |      Coef. | Std. Error |     t | Pr(>|t|) |  Lower 95% |  Upper 95% |
|:------------|-----------:|-----------:|------:|:---------|:-----------|-----------:|
| (Intercept) |  0.0795487 |  0.0648201 |  1.23 |   0.2200 | -0.0476512 |  0.206749 |
| D           |  1.98177   |  0.0653341 | 30.33 |   <1e-99 |  1.85357   |  2.10998 |
| X1          |  0.48678   |  0.0330758 | 14.72 |   <1e-43 |  0.421873  |  0.551686 |
| X2          | -0.319956  |  0.0653371 | -4.90 |   <1e-05 | -0.448171  | -0.191742 |
| X3          |  0.156724  |  0.0321395 |  4.88 |   <1e-05 |  0.0936552 |  0.219793 |
| X4          | -0.10825   |  0.0653549 | -1.66 |   0.0980 | -0.236499  |  0.0199992 |

Coef. D (ATE): 1.982
SE:            0.065


In [22]:
println("=== COMPARACIÓN ===")
println("Diferencia en ATE: ", round(β_D_comp - β_D_simple, digits=3))
println("Diferencia en SE:  ", round(se_D_comp - se_D_simple, digits=3))


=== COMPARACIÓN ===
Diferencia en ATE: 0.019
Diferencia en SE:  -0.008


In [38]:
using Statistics, Distributions, DataFrames, GLM, StatsModels, LinearAlgebra

soft_threshold(z, γ) = sign(z) * max(abs(z) - γ, 0.0)

function lasso_cd(X::AbstractMatrix, y::AbstractVector; λ::Float64,
                  fit_intercept::Bool=true, standardize::Bool=false,
                  maxiter::Int=5_000, tol::Float64=1e-6)
    Xc = copy(X); yc = copy(y)
    μx = zeros(size(X,2)); σx = ones(size(X,2))
    μy = 0.0

    if standardize
        for j in 1:size(X,2)
            μx[j] = mean(Xc[:,j]); σx[j] = std(Xc[:,j])
            σx[j] > 0 ? (Xc[:,j] .-= μx[j]; Xc[:,j] ./= σx[j]) : nothing
        end
    end
    if fit_intercept
        μy = mean(yc); yc .-= μy
    end

    n, p = size(Xc)
    β = zeros(p)
    r = copy(yc)

    @views xnorm2 = [sum(Xc[:, j].^2) for j in 1:p]

    for _ in 1:maxiter
        maxchg = 0.0
        @views for j in 1:p
            r .+= β[j] .* Xc[:, j]
            ρ = dot(Xc[:, j], r)
            β_new = soft_threshold(ρ, λ) / xnorm2[j]
            r .-= β_new .* Xc[:, j]
            maxchg = max(maxchg, abs(β_new - β[j]))
            β[j] = β_new
        end
        if maxchg < tol
            break
        end
    end

    β_out = copy(β); α = 0.0
    if standardize
        β_out .= β_out ./ σx
        if fit_intercept
            α = μy - sum(μx .* β_out)
        end
    else
        if fit_intercept
            α = μy
        end
    end
    return β_out, α
end

# --- prepara datos ---
X = Matrix(df[:, [:X1, :X2, :X3, :X4]])
y = df.Y

Xstd = copy(X)
for j in (1, 3)
    μ = mean(X[:, j]); σ = std(X[:, j])
    σ > 0 ? (Xstd[:, j] .= (X[:, j] .- μ) ./ σ) : nothing
end

chernozhukov_lambda(n, p; confidence=0.95) = begin
    c = 1.1; α = 1 - confidence; z = quantile(Normal(), 1 - α/(2p))
    c * sqrt(n) * z
end

nX, pX = size(Xstd)
λ_ch  = chernozhukov_lambda(nX, pX)
λ_use = λ_ch / nX

β_theo, α_theo = lasso_cd(Xstd, y; λ=λ_use, fit_intercept=true, standardize=false)

pnames = [:X1, :X2, :X3, :X4]
coef_theo_df = DataFrame(Variable=pnames, Coeficiente=β_theo)
coef_theo_df.Abs = abs.(coef_theo_df.Coeficiente)
sort!(coef_theo_df, :Abs, rev=true)

println("\n=== LASSO TEÓRICO (CD) ===")
println("λ (crudo): ", λ_ch, " | λ/n: ", λ_use)
display(coef_theo_df)

ε = 1e-8
vars_sel_theo = vec(coef_theo_df[abs.(coef_theo_df.Coeficiente) .> ε, :Variable])
println("Seleccionadas (teórico): ", vars_sel_theo)



=== LASSO TEÓRICO (CD) ===
λ (crudo): 86.88282045755983 | λ/n: 0.08688282045755984


Row,Variable,Coeficiente,Abs
,Symbol,Float64,Float64
1,X1,0.477602,0.477602
2,X3,0.161351,0.161351
3,X2,-0.12236,0.12236
4,X4,0.0303106,0.0303106


Seleccionadas (teórico): [:X1, :X3, :X2, :X4]


In [40]:
using Random

function cv_lasso(X, y; k::Int=5,
                  λ_grid::AbstractVector=exp10.(range(log10(λ_use*5), log10(λ_use/200), length=40)),
                  seed::Int=42)
    Random.seed!(seed)
    n = size(X,1)
    idx = shuffle(1:n)
    folds = [ idx[floor(Int,(i-1)*n/k)+1 : floor(Int,i*n/k)] for i in 1:k ]

    mse = zeros(length(λ_grid))
    for (g, λ) in pairs(λ_grid)
        err = 0.0
        for f in 1:k
            test = folds[f]; train = setdiff(1:n, test)
            β, α = lasso_cd(X[train, :], y[train]; λ=λ, fit_intercept=true, standardize=false)
            yhat = α .+ X[test, :]*β
            err += mean((y[test] .- yhat).^2)
        end
        mse[g] = err / k
    end
    j = argmin(mse)
    return λ_grid[j], mse[j], λ_grid, mse
end

λ_cv, mse_cv, grid, grid_mse = cv_lasso(Xstd, y; k=5)
β_cv, α_cv = lasso_cd(Xstd, y; λ=λ_cv, fit_intercept=true, standardize=false)

coef_cv_df = DataFrame(Variable=pnames, Coeficiente=β_cv)
coef_cv_df.Abs = abs.(coef_cv_df.Coeficiente)
sort!(coef_cv_df, :Abs, rev=true)

println("\n=== LASSO CV (CD) ===")
println("λ óptimo CV: ", λ_cv)
display(coef_cv_df)

vars_sel_cv = vec(coef_cv_df[abs.(coef_cv_df.Coeficiente) .> ε, :Variable])
println("Seleccionadas (CV): ", vars_sel_cv)

# Re-estimación GLM
function formula_from_vars(vars::Vector{Symbol})
    isempty(vars) ? @formula(Y ~ D) : Term(:Y) ~ sum(Term.([:D; vars]))
end

f_theo = formula_from_vars(vars_sel_theo)
f_cv   = formula_from_vars(vars_sel_cv)

modelo_theo = lm(f_theo, df)
modelo_cv   = lm(f_cv, df)

println("\n=== REGRESIONES RESTRINGIDAS ===")
println("Teórico:"); display(GLM.coeftable(modelo_theo))
println("CV:");      display(GLM.coeftable(modelo_cv))

β_D_theo = coef(modelo_theo)[2]
β_D_cv   = coef(modelo_cv)[2]

println("\n=== COMPARACIÓN FINAL DE ATE ===")
println("ATE verdadero: 2.000")
println("ATE teórico:   ", round(β_D_theo,   digits=3))
println("ATE CV:        ", round(β_D_cv,     digits=3))
println("ATE completo:  ", round(β_D_comp,   digits=3))
println("ATE simple:    ", round(β_D_simple, digits=3))

println("\n=== Escalas de λ ===")
println("λ CV: ", λ_cv, " | λ Chernozhukov (λ/n): ", λ_use, " | Ratio CV/Ch: ", λ_cv/λ_use)



=== LASSO CV (CD) ===
λ óptimo CV: 0.4344141022877992


Row,Variable,Coeficiente,Abs
,Symbol,Float64,Float64
1,X1,0.477284,0.477284
2,X3,0.161019,0.161019
3,X2,-0.120964,0.120964
4,X4,0.028941,0.028941


Seleccionadas (CV): [:X1, :X3, :X2, :X4]

=== REGRESIONES RESTRINGIDAS ===
Teórico:


|             |      Coef. | Std. Error |     t | Pr(>|t|) |  Lower 95% |  Upper 95% |
|:------------|-----------:|-----------:|------:|:---------|:-----------|-----------:|
| (Intercept) |  0.0795487 |  0.0648201 |  1.23 |   0.2200 | -0.0476512 |  0.206749 |
| D           |  1.98177   |  0.0653341 | 30.33 |   <1e-99 |  1.85357   |  2.10998 |
| X1          |  0.48678   |  0.0330758 | 14.72 |   <1e-43 |  0.421873  |  0.551686 |
| X3          |  0.156724  |  0.0321395 |  4.88 |   <1e-05 |  0.0936552 |  0.219793 |
| X2          | -0.319956  |  0.0653371 | -4.90 |   <1e-05 | -0.448171  | -0.191742 |
| X4          | -0.10825   |  0.0653549 | -1.66 |   0.0980 | -0.236499  |  0.0199992 |

CV:


|             |      Coef. | Std. Error |     t | Pr(>|t|) |  Lower 95% |  Upper 95% |
|:------------|-----------:|-----------:|------:|:---------|:-----------|-----------:|
| (Intercept) |  0.0795487 |  0.0648201 |  1.23 |   0.2200 | -0.0476512 |  0.206749 |
| D           |  1.98177   |  0.0653341 | 30.33 |   <1e-99 |  1.85357   |  2.10998 |
| X1          |  0.48678   |  0.0330758 | 14.72 |   <1e-43 |  0.421873  |  0.551686 |
| X3          |  0.156724  |  0.0321395 |  4.88 |   <1e-05 |  0.0936552 |  0.219793 |
| X2          | -0.319956  |  0.0653371 | -4.90 |   <1e-05 | -0.448171  | -0.191742 |
| X4          | -0.10825   |  0.0653549 | -1.66 |   0.0980 | -0.236499  |  0.0199992 |


=== COMPARACIÓN FINAL DE ATE ===
ATE verdadero: 2.000
ATE teórico:   1.982
ATE CV:        1.982
ATE completo:  1.982
ATE simple:    1.963

=== Escalas de λ ===
λ CV: 0.4344141022877992 | λ Chernozhukov (λ/n): 0.08688282045755984 | Ratio CV/Ch: 5.0
